In [7]:
import sys
sys.path.append("../bus_service_increase") 
sys.path.append("../starter_kit") # to test out style_df function

In [11]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from IPython.display import display, HTML, Markdown

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import PUBLIC_FILENAME, YEAR, MONTH
from explore_monthly_ridership_by_rtpa import sum_by_group
from _starterkit_utils import style_df
#from monthly_ridership_by_rtpa import get_percent_change
#from shared_utils.rt_dates import MONTH_DICT

#Temp file path for testing
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

#alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [36]:
pd.set_option("display.max_rows", None)

In [9]:
# parameters cell for local
rtpa = "Metropolitan Transportation Commission" # lots of reporters
#rtpa = "Butte County Association of Governments" # 2 reporters 
#rtpa = "San Joaquin Council of Governments" # 4 reporters

In [10]:
%%capture_parameters
rtpa

{"rtpa": "Metropolitan Transportation Commission"}


---

## Update `produce_ntd_monthly_ridership_by_rtpa` function

In [17]:
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, filter, show_query, select

In [31]:
# function to make crosswalk. to be used for other analyses
def make_ntd_id_rtpa_crosswalk(socal_split:bool):
    """
    Function that reads in the RTPA data from dim_orgs. joins in the county data from bridge table.
    Option to split out the socal counties into the county ctc level instead of just SCAG.
    """
    
    socal_county_dict = {
        "Ventura": "Ventura County Transportation Commission",
        "Los Angeles": "Los Angeles County Metropolitan Transportation Authority",
        "San Bernardino": "San Bernardino County Transportation Authority",
        "Riverside": "Riverside County Transportation Commission",
        "Orange": "Orange County Transportation Authority",
    }
    
    # get rtpa_name from dim_orgs
    dim_orgs = (
        tbls.mart_transit_database.dim_organizations()
        >> filter(
            _._is_current == True,
            _.ntd_id_2022.notna(),
            _.rtpa_name.notna(),
        )
        >> select(
            _.name, 
            _.ntd_id_2022, 
            _.rtpa_name, 
            _.mpo_name, 
            _.key)
        >> collect()
    )
    
    # get county data
    bridge_counties = (
        tbls.mart_transit_database.bridge_organizations_x_headquarters_county_geography()
        >> filter(_._is_current == True)
        >> select(_.county_geography_name, _.organization_key)
        >> collect()
    )
    
    # merge dim_org to county
    crosswalk = dim_orgs.merge(
        bridge_counties, left_on="key", right_on="organization_key", how="left"
    )
    
    if socal_split is True:
        crosswalk.loc[crosswalk["county_geography_name"].isin(socal_county_dict.keys()), "rtpa_name"] = (
        crosswalk["county_geography_name"].map(socal_county_dict)
    )
    
    return crosswalk

In [32]:
test = make_ntd_id_rtpa_crosswalk(socal_split=False)

In [33]:
display(
    test.head(),
    test.info(),
    test["rtpa_name"].value_counts()
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   244 non-null    object
 1   ntd_id_2022            244 non-null    object
 2   rtpa_name              244 non-null    object
 3   mpo_name               161 non-null    object
 4   key                    244 non-null    object
 5   county_geography_name  244 non-null    object
 6   organization_key       244 non-null    object
dtypes: object(7)
memory usage: 15.2+ KB


,name,ntd_id_2022,rtpa_name,mpo_name,key,county_geography_name,organization_key
0,Access Services,90157,Southern California Association of Governments,None,d84a961daa618c733f9d9c3bd49c322f,Los Angeles,d84a961daa618c733f9d9c3bd49c322f
1,Alpine County,91116,Alpine County Local Transportation Commission,Tahoe Regional Planning Agency,9b5971d16d58e4fcafa694ee7fa33b12,Alpine,9b5971d16d58e4fcafa694ee7fa33b12
2,Bishop Paiute Tribe,99268,Inyo County Local Transportation Commission,None,957618c89db2f5e992caa5ca2e6086ab,Inyo,957618c89db2f5e992caa5ca2e6086ab
3,Calaveras County,91063,Calaveras Council of Governments,None,a024fabd0002f9c9bd636042de30715d,Calaveras,a024fabd0002f9c9bd636042de30715d
4,California Vanpool Authority,90230,Kings County Association of Governments,None,e0198bfce62fdfd4f0b7af46f5585069,Kings,e0198bfce62fdfd4f0b7af46f5585069


None

Southern California Association of Governments          93
Metropolitan Transportation Commission                  29
Kern Council of Governments                             11
San Diego Association of Governments                    10
Sacramento Area Council of Governments                  10
Tulare County Association of Governments                 8
Ventura County Transportation Commission                 8
Santa Barbara County Association of Governments          7
San Joaquin Council of Governments                       7
Stanislaus Council of Governments                        5
Placer County Transportation Planning Agency             4
Madera County Transportation Commission                  4
Humboldt County Association of Governments               4
Merced County Association of Governments                 3
Kings County Association of Governments                  3
Del Norte Local Transportation Commission                3
Fresno Council of Governments                           

In [26]:
test2 = make_ntd_id_rtpa_crosswalk(socal_split=True)

In [28]:
test["rtpa_name"].unique() == test2["rtpa_name"].unique()

False

In [ ]:
def produce_ntd_monthly_ridership_by_rtpa(
    year: int,
    month: int
) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    Import NTD data from warehouse, filter to CA, 
    merge in crosswalk, checks for unmerged rows, then creates new columns for full Mode and TOS name.
    
    """
    
    full_upt = (
        tbls.mart_ntd.dim_monthly_ridership_with_adjustments() 
        >> collect()
    ).rename(
        columns = {
            "mode_type_of_service_status": "Status",
            "primary_uza_name":"uza_name"
        })
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    
    full_upt.to_parquet(
        f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    )
    
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    # use new crosswalk function
    crosswalk = make_ntd_id_rtpa_crosswalk(socal_split=True)
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        left_on = "ntd_id",
        right_on ="ntd_id_2022"
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    df = add_change_columns(df)
    
    df = df.assign(
        Mode_full = df["mode"].map(NTD_MODES),
        TOS_full = df["tos"].map(NTD_TOS)
    )
    
    return df

---

# {rtpa}
## Monthly Ridership Trends

**Download data from our public [folder](https://console.cloud.google.com/storage/browser/calitp-publish-data-analysis)** by navigating to `ntd_monthly_ridership` and selecting a file.

Transit operators/agencies that are **Urban full reporters, that submit monthly ridership data to NTD from 2018 to present**, are included in this report. Reporters that were previously Urban full reporters, but are currently not, may appear. This may result in Reporters showing zero or partial ridership data in the report. 

If a Reporter is not a monthly reporter, or has not reported data since 2018, they will not appear in the report.

Examples: 
- Reporter A is an urban full reporter from 2019-2022, then became a reduced reporter for 2023. Reporter A's ridership data will be displayed for 2019-2022 only.
- Reporter B is an urban full reporter from 2000-2017, then became a reduced reporter for 2018. Reporter B will be named in the report, but will not display ridership data.
- Reporter C was a reduced reporter form 2015-2020, then became an urban full reporter and began submitting monthly ridership data to NTD for 2021. Reporter C's ridership data will be displayed for 2021-present.

In [ ]:
URL = ("https://console.cloud.google.com/storage/"
       "browser/calitp-publish-data-analysis"
      )

display(
    HTML(
        f"""
        <a href={URL}>
        Download the latest month of data: {PUBLIC_FILENAME}</a>
        """
    )
)


In [ ]:
MIN_YEAR = 2018

#updated to filter for period_year greater than 2018
df = pd.read_parquet(
    f"{GCS_FILE_PATH}ca_monthly_ridership_{YEAR}_{MONTH}.parquet",
    filters = [[("RTPA", "==", rtpa), ("period_year", ">=", MIN_YEAR)]]
).drop(
    #updated columns names to match new df
    columns = ["mode", "tos","execution_ts"]
).rename(columns = {"Mode_full": "Mode", "TOS_full": "TOS"})

In [ ]:
df.info()

In [ ]:
#REMOVE

# find columns that are recent enough to plot
#MIN_YEAR = 2018

#might not need the rest of these
#not_id_cols = [c for c in df.columns if "/" in c]

#recent_years = [
#    c for c in not_id_cols if int(c.split("/")[1]) >= MIN_YEAR and 
#    "pct" not in c
#]

#upt_cols = [
#    c for c in recent_years if "change" not in c
#]

#change_cols = [c for c in recent_years if "change" in c]

In [ ]:
#REMOVE

# do i need this anymore? 
# what does the data look like initially? filters/groups the DF by the `group_cols` list, keeps the initial wide data

# give this a try with current data NOPE! 
#def OLD_sum_by_group(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
#    """
#    Since df is wide, use pivot_table() to sum up all
#    the columns that show UPT.
#    """
#    grouped_df = df.pivot_table(
#        index = group_cols, 
#        values = "upt", 
#        aggfunc="sum"
#    ).reset_index().reindex(columns = group_cols + recent_years)
    
#    return grouped_df


# DONT THINK I NEED THIS ANYMORE!!!
#def make_long(df: pd.DataFrame, group_cols: list, value_cols: list):
#    df_long = df[group_cols + value_cols].melt(
#        id_vars = group_cols, 
#        value_vars = value_cols,
#    )
    
#    df_long = df_long.assign(
#        variable = df_long.variable.str.replace("change_1yr_", "")
#    )
    
#    return df_long

In [ ]:
## MOVED FUNCTION TO SCRIPT, SO IT CAN BE USED BY save_rtpa_outputs

#function is read in with imports

# UPDATED AND TESTED, this works! gives the same results as the old old sum_by_group, make_long and assemble_long_df functions

#def sum_by_group(
#    df: pd.DataFrame,
#    group_cols: list) -> pd.DataFrame:
#    """
#    since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
#    """
#    from TEST_monthly_ridership_by_rtpa import get_percent_change
#    grouped_df = df.groupby(group_cols+
#                             ['period_year',
#                             'period_month',
#                             'period_year_month']
#                           ).agg({
#        "upt":"sum",
#        "previous_y_m_upt":"sum",
#        "change_1yr":"sum"
#    }
#    ).reset_index()
    
    #get %change back
#    grouped_df = get_percent_change(grouped_df)
    
    #decimal to whole number
#    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
#    return grouped_df
    

In [ ]:
#REMOVE

# takes the sum_by_group dfs and makes them long
# unpivots the sum_by_group dfs, each row is a different year/month

# DONT THINK I NEED THIS ANYMORE!!!
#def assemble_long_df(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
#    """
#    Need df to be long to make chart.
#    Let's put raw UPT and change side-by-side.
#    """
#    df_raw = make_long(df, group_cols, upt_cols).rename(
#        columns = {"value": "upt"})
#    df_change = make_long(df, group_cols, change_cols).rename(
#        columns = {"value": "change_1yr"})

#    final = pd.merge(
#        df_raw,
#        df_change,
#        on = group_cols + ["variable"],
#        how = "left"
#    )
    
#    final = final.assign(
#        year = final.variable.str.split("/", expand=True)[1],
#        month = final.variable.str.split("/", expand=True)[0].str.zfill(2)
#    )
    
#    final = final.assign(
#        year_month = final.year + "-" + final.month
#    )
    
#    return final

In [ ]:
# KEEPING AS IS FOR NOW, BUT MAY NEED TO REWORK 

def remove_zero_upt_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    takes the by_agency_long df, filters for rows with zero UPT AND zero change_1yr. 
    then removes these rows from the by_agency_long df.
    resulting df should only contain rows with change in UPT not due to (0 UPT - 0 UPT) and net zero change in UPT
    zero UPT and zero change_1yr occurs when (0 upt - 0upt = 0change), dont need to show this
    but a net zero change (100upt - 100upt = 0 change), should be shown
    """
    
    #df of rows with zero UPT for 2 years
    zero_UPT_2years = df[(df["upt"] == 0) & (df["change_1yr"] == 0)]
    
    merge = df.merge(zero_UPT_2years, how="left", indicator=True)
    no_zero_UPT_rows = merge[merge["_merge"] == "left_only"].drop(columns=["_merge"]).dropna(subset="change_1yr")
    
    return no_zero_UPT_rows
    

In [ ]:
def group_by_agency(df):
    """
    Take in the 'by_ageny_long' df and aggregatese by rtpa, and calculates upt % of total. 
    To be used in pie chart
    """
    initial_agg = df.groupby("agency").agg(
        total_upt=("upt","sum")
    ).reset_index()
    
    # % total columns
    initial_agg["pct_of_total_upt"] = (initial_agg["total_upt"]/initial_agg["total_upt"].sum())*100
    
    # cleaning data types and rounding
    initial_agg["total_upt"] = initial_agg["total_upt"].astype("int64")
    initial_agg["pct_of_total_upt"] = initial_agg["pct_of_total_upt"].round(decimals=2)
    cleaned_agg = initial_agg.sort_values(by="total_upt", ascending = False)
    
    return cleaned_agg

In [ ]:
# NEW 

agency_cols = ["ntd_id", "agency", "RTPA"]
mode_cols = ["Mode", "RTPA"]
tos_cols = ["TOS", "RTPA"]

# sum_by_group still works! now that it is imported!
by_agency_long = sum_by_group(df, agency_cols)
by_mode_long = sum_by_group(df, mode_cols)
by_tos_long = sum_by_group(df, tos_cols)

# zero upt still works!
by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

# total UPT check 

total_upt = by_agency_long["upt"].sum()
agency_count = by_agency_long["agency"].nunique()


agency_agg_yr = group_by_agency(by_agency_long)


In [ ]:
#REMOVE

#I STILL NEED FILTERED df BY AGENCY, MODE AND TOS

#by_agency_long = assemble_long_df(by_agency, agency_cols)
#by_mode_long = assemble_long_df(by_mode, mode_cols)
#by_tos_long = assemble_long_df(by_tos, tos_cols)

#by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

In [ ]:
LABELING_DICT = {
    "upt": "Unlinked Passenger Trips",
    "change_1yr": "Change in Unlinked Passenger Trips from Prior Year",
    "TOS": "Type of Service",
    "year_month": "Date"
}

def labeling(word: str) -> str:
    return chart_utils.labeling(word, LABELING_DICT)

WIDTH = 300
HEIGHT = 150

In [ ]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
        
    return chart

In [ ]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
    
    return chart

In [ ]:
### initial pie code
def make_pie_chart(df,
             col,
             color_col):
    
    pie = alt.Chart(df).mark_arc(radius = 150).encode(
        theta=col,
        color=color_col,
        tooltip = ["agency","total_upt","pct_of_total_upt"]
    ).properties(
        title=f"Total Unlinked Passenger Trips per agency in RTPA since {MIN_YEAR}",

    )

    return pie

In [ ]:
## moved to prod notebook 1/15/2025
#simple bar chart for total agencies and UPT
def total_upt_chart(df: pd.DataFrame, x_col:str, y_col:str,tool_tip:list):
    bar_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(x_col).sort("-y"),
    y=alt.Y(y_col),
    tooltip = tool_tip,
    color = alt.Color(x_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               ))
    ).properties(
    title=f"Total Unlinked Passenger Trips per agency in RTPA since {MIN_YEAR}",
    width = WIDTH, 
    height = HEIGHT,
    ).resolve_scale(y="independent").interactive()
    
    return bar_chart

### Totals

In [ ]:
Markdown(f"""
Within {rtpa}:
- Number of Agencies/Operators: <b>{agency_count}</b>.
- Total Unlinked Passenger Trips since {MIN_YEAR}: <b>{total_upt:,}</b>.
- Individual agency/operator ridership breakdown:
""")

In [ ]:
display(agency_agg_yr.reset_index(drop=True))

In [ ]:
df_html= agency_agg_yr.reset_index(drop=True).to_html()

print(df_html)

In [ ]:
df_markdown= agency_agg_yr.reset_index(drop=True).to_markdown()

print(df_markdown)

In [ ]:
tooltip_list = ["agency","total_upt","pct_of_total_upt"]

total_upt_chart(
    agency_agg_yr,
    x_col="agency", 
    y_col="total_upt", 
    tool_tip=tooltip_list)

In [ ]:
make_pie_chart(agency_agg_yr, col = "total_upt", color_col="agency")


### Transit Agency

In [ ]:
make_line_chart(by_agency_long, y_col = "upt", color_col = "agency")

Change in Unlinked Passenger Trips from the prior year. For example, July 2023's change would be the change in July 2023's reported values against July 2022's reported values.

In [ ]:
# TEST OF BAR CHART WITH % CHANGE 1 YEAR
make_bar_chart(by_agency_long_no_zero_upt[by_agency_long_no_zero_upt["period_year"] >= 2018], 
               y_col = "pct_change_1yr", 
               color_col = "agency")

In [ ]:
make_bar_chart(by_agency_long_no_zero_upt[by_agency_long_no_zero_upt["period_year"] >= 2024], y_col = "change_1yr", color_col = "agency")

In [ ]:
make_bar_chart(by_agency_long_no_zero_upt, y_col = "change_1yr", color_col = "agency")

### Transit Mode

In [ ]:
make_line_chart(by_mode_long, y_col = "upt", color_col = "Mode")

In [ ]:
make_bar_chart(by_mode_long[by_mode_long["period_year"] >= 2024], y_col = "change_1yr", color_col = "Mode")

### Type of Service

In [ ]:
make_line_chart(by_tos_long, y_col = "upt", color_col = "TOS")

In [ ]:
make_bar_chart(by_tos_long[by_tos_long["period_year"] >= 2024], y_col = "change_1yr", color_col = "TOS")